# Creating annual vegetation curves to correlate with ground-based surveys

This code processes geospatial data to analyze and visualize the clear sky conditions, NDVI, and LAI indices for specified sites over different years. It uses a shapefile to define the study areas and Earth Engine to retrieve satellite imagery. For each site and year, the code calculates clear sky percentages, NDVI, LAI, and aerosol optical thickness (AOT). It identifies the closest dates to the end of specified survey periods with varying levels of clear sky coverage and saves these dates in a CSV file. The code also generates scatter plots of NDVI and LAI indices, highlighting the closest date with over 99% clear sky pixels in yellow, and saves these plots in the working directory. This comprehensive analysis helps in understanding the vegetation dynamics and atmospheric conditions during the survey periods.

Follow-up code to this will provide functionalitly to download the "closest clear-sky" image for each site for processing in SNAP, or with success, automatically download and process that imagery using SNAP algorithms via snappy or satellitetools packages 

Load required packages 

In [ ]:
import ee
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
from datetime import datetime
import os

Authenticate and initialize google earth engine - you will need an account. If it is your first time linking to your account, a new window will open and take you through the process. If this does not work, try ee.Authenticate(force=True). 

In [ ]:
# Initialize the Earth Engine module.
ee.Authenticate()
ee.Initialize()


Load a shapefile, convert its coordinates to WGS84 projection, extract polygon geometries, and ensure they are valid for use with Google Earth Engine.

In [ ]:
# Load the shapefile
shapefile_path = r'C:\Your shapefile.shp'
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs(epsg=4326)  # Convert to WGS84 projection
geojson = gdf.__geo_interface__
site_names = gdf['Site'].tolist() #Change this to correspond to your polygon id 

# Convert GeoJSON to EE geometries and ensure valid polygons
polygons = []
for feature in geojson['features']:
    if feature['geometry']['type'] == 'Polygon':
        for coords in feature['geometry']['coordinates']:
            if len(coords) >= 3:  # Ensure there are at least 3 points
                polygons.append(ee.Geometry.Polygon(coords))
    elif feature['geometry']['type'] == 'MultiPolygon':
        for polygon in feature['geometry']['coordinates']:
            for coords in polygon:
                if len(coords) >= 3:  # Ensure there are at least 3 points
                    polygons.append(ee.Geometry.Polygon(coords))


Enter survey dates or date ranges and convert to pandas datetime format

In [ ]:
# Define survey date ranges for each year
survey_ranges = {
    2019: ('2019-07-26', '2019-07-27'),
    2020: ('2020-07-06', '2020-07-09'),
    2021: ('2021-07-10', '2021-07-12'),
    2022: ('2022-07-01', '2022-07-02'),
    2023: ('2023-07-24', '2023-07-30'),
}

# Convert survey ranges to pandas datetime format
for year in survey_ranges:
    survey_ranges[year] = (pd.to_datetime(survey_ranges[year][0]), pd.to_datetime(survey_ranges[year][1]))

The s2_clear_sky function applies a mask to satellite imagery to retain only the pixels classified as clear sky using the SCL band, following the s2ClearSky method. 

The calculate_clear_sky_percentage function then calculates the percentage of clear sky pixels within a specified geometry by comparing the count of clear sky pixels to the total number of pixels in the region.

In [ ]:
# Cloud masking function using the SCL band following the s2ClearSky method
def s2_clear_sky(image):
    scl = image.select('SCL')
    clear_sky_pixels = scl.eq(4).Or(scl.eq(5)).Or(scl.eq(6)).Or(scl.eq(11)) #This means that only veg (4) bare soil (5) water (6) and snow (11) pixels are considered "clear sky" 
    return image.updateMask(clear_sky_pixels) 

# Function to calculate clear sky percentage using the SCL band
def calculate_clear_sky_percentage(image, geometry):
    total_pixels = image.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=geometry,
        scale=30,  # Use larger scale to reduce computation decrease to 10 for reporting on small <10ha areas
        bestEffort=True
    ).values().get(0)
    
    clear_sky_pixels = s2_clear_sky(image).reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=geometry,
        scale=30,  # Use larger scale to reduce computation decrease to 10 for reporting on small <10ha areas
        bestEffort=True
    ).values().get(0)
    
    clear_sky_percentage = ee.Number(clear_sky_pixels).divide(total_pixels).multiply(100)
    return clear_sky_percentage



This code processes geospatial data to analyze clear sky conditions, NDVI, LAI, and AOT indices for specified sites over different years, generating plots and saving results for further analysis.
- NDVI (Normalized Difference Vegetation Index): NDVI is a measure of vegetation health and density, calculated using the difference between near-infrared and red light reflected by vegetation.
- LAI (Leaf Area Index): LAI quantifies the total leaf area of plants per unit ground area, indicating the density of plant foliage.
- AOT (Aerosol Optical Thickness): AOT measures the extent to which aerosols prevent the transmission of sunlight through the atmosphere, indicating the concentration of particulate matter. If applicable, this can be used to filter images effected by wildfire smoke. 

In [ ]:
# Function to calculate AOT, NDVI, and LAI, filtering out non-clear sky pixels
def calculate_indices(image, geometry):
    # Apply clear sky mask
    clear_sky_image = s2_clear_sky(image)
    
    # Calculate NDVI
    ndvi = clear_sky_image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    # Calculate LAI
    lai = clear_sky_image.expression('3.618 * NDVI - 0.118', {'NDVI': ndvi}).rename('LAI') # The LAI VI is of questionable use - but the goal of this program is LAI, though we will use SNAP from the closest_date image
    # Calculate AOT using Sentinel-5P
    start_date = image.date().advance(-1, 'day')
    end_date = image.date().advance(1, 'day')
    aot = ee.ImageCollection('COPERNICUS/S5P/OFFL/L3_AER_AI') \
        .filterDate(start_date, end_date) \
        .filterBounds(geometry) \
        .mean().select('absorbing_aerosol_index')
    
    # Reduce to region
    ndvi_mean = ndvi.reduceRegion(reducer=ee.Reducer.mean(), geometry=geometry, scale=30, bestEffort=True).get('NDVI') # Use larger scale to reduce computation decrease to 10 for reporting on small <10ha areas
    lai_mean = lai.reduceRegion(reducer=ee.Reducer.mean(), geometry=geometry, scale=30, bestEffort=True).get('LAI') # Use larger scale to reduce computation decrease to 10 for reporting on small <10ha areas
    aot_mean = aot.reduceRegion(reducer=ee.Reducer.mean(), geometry=geometry, scale=1000, bestEffort=True).get('absorbing_aerosol_index') # this scale means it can't be used for a mask, only dropping returns 

    return ndvi_mean, lai_mean, aot_mean


This section processes geospatial data for each site and year to calculate clear sky percentages, NDVI, LAI, and AOT indices, and stores the results in data frames. It divides the year into batches to handle satellite imagery, computes relevant indices, and saves the data to CSV files. Additionally, it identifies and records the closest dates to the end of specified survey periods that meet varying clear sky thresholds.

In [ ]:
# Function to get clear sky data for each year in batches
def process_year_data(year, geometry, site_name):
    start_date = f'{year}-04-01' # Change these dates depending on your growing season
    end_date = f'{year}-10-31'
    
    def get_clear_sky_data(image):
        date = image.date().format('YYYY-MM-dd')
        clear_sky_percentage = calculate_clear_sky_percentage(image, geometry)
        ndvi_mean, lai_mean, aot_mean = calculate_indices(image, geometry)
        return ee.Feature(None, {
            'date': date,
            'clear_sky_percentage': clear_sky_percentage,
            'ndvi': ndvi_mean,
            'lai': lai_mean,
            'aot': aot_mean
        })

    # Process in batches
    date_ranges = [
        (f'{year}-04-01', f'{year}-05-31'),
        (f'{year}-06-01', f'{year}-07-31'),
        (f'{year}-08-01', f'{year}-09-30'),
        (f'{year}-10-01', f'{year}-10-31')
    ]
    
    data = []
    for start, end in date_ranges:
        sentinel_collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
            .filterDate(start, end) \
            .filterBounds(geometry)
        
        clear_sky_data = sentinel_collection.map(get_clear_sky_data).getInfo()
        
        for feature in clear_sky_data['features']:
            date = feature['properties']['date']
            clear_sky_percentage = feature['properties']['clear_sky_percentage']
            ndvi = feature['properties']['ndvi']
            lai = feature['properties']['lai']
            aot = feature['properties']['aot'] #drop AOT if you aren't worried about smoke/pollution
            data.append((date, clear_sky_percentage, ndvi, lai, aot))
    
    df = pd.DataFrame(data, columns=['Date', 'Clear_Sky_Percentage', 'NDVI', 'LAI', 'AOT'])
    df['Date'] = pd.to_datetime(df['Date'])  # Ensure 'Date' column is of datetime type
    
    # Save the DataFrame as a CSV file in the same folder as the shapefile
    csv_path = os.path.join(os.path.dirname(shapefile_path), f'{site_name}_{year}_data.csv')
    df.to_csv(csv_path, index=False)
    
    # Find the closest date to the survey end date with different clear sky thresholds - you can drop the thresholds if your area of interest is less cloudy
    end_date = survey_ranges[year][1]
    closest_dates = {}
    for threshold in [80, 90, 95, 99]:
        df_filtered = df[df['Clear_Sky_Percentage'] >= threshold]
        if not df_filtered.empty:
            closest_date = df_filtered.iloc[(df_filtered['Date'] - end_date).abs().argsort()[:1]]['Date'].values[0]
            closest_dates[threshold] = closest_date
        else:
            closest_dates[threshold] = None
    
    print(f"Year {year} for site {site_name}: Closest dates to end of survey")
    for threshold, date in closest_dates.items():
        print(f"{threshold}% clear sky: {date}")
    
    return df, closest_dates



This section runs the functions defined above to processes satellite imagery data for specified sites and years to calculate clear sky percentages, NDVI, LAI, and AOT indices. It identifies and records the closest dates with different clear sky thresholds to the end of survey periods, saving these dates in a CSV file. 

In [ ]:
years = [2023]  # Adjust the years as needed

# Store data frames and closest dates for each year and site
all_data_frames = {}
closest_dates_all_sites = []

# Process by polygon (first "for") - Pull indicies (second "for") - Pull closest dates (third "for") 
for i, (polygon, site_name) in enumerate(zip(polygons, site_names)):
    print(f"Processing polygon {i+1}/{len(polygons)} for site {site_name}")
    
    try:
        for year in years:
            df, closest_dates = process_year_data(year, polygon, site_name)
            all_data_frames[(site_name, year)] = df
            for threshold, date in closest_dates.items():
                closest_dates_all_sites.append({
                    'Site': site_name,
                    'Year': year,
                    'Threshold': threshold,
                    'Date': date
                })
    
    except ee.ee_exception.EEException as e:
        print(f"Error processing polygon {i+1} for site {site_name}: {e}")

# Save the closest dates to a separate CSV file from the site/year indicies
closest_dates_df = pd.DataFrame(closest_dates_all_sites)
closest_dates_csv_path = os.path.join(os.path.dirname(shapefile_path), 'closest_dates_to_survey_end.csv')
closest_dates_df.to_csv(closest_dates_csv_path, index=False)


This code generates scatter plots of the NDVI and LAI indices, highlighting the closest date with over 99% clear sky pixels, and saves these plots as images in the working directory. 

In [ ]:
def plot_indices(df, site_name, year, index, survey_start, survey_end, closest_date_99):
    plt.figure(figsize=(10, 6))

    df = df[df['Clear_Sky_Percentage'] >= 90]
    
    survey_period = (df['Date'] >= survey_start) & (df['Date'] <= survey_end)
    x = pd.to_datetime(df['Date'])
    y = df[index]
    
    plt.scatter(x[~survey_period], y[~survey_period], label=f'{site_name} {index}', alpha=0.6, color='cornflowerblue')
    
    non_survey_data = df[~survey_period]
    if not non_survey_data.empty:
        non_survey_data = non_survey_data.sort_values('Date')
        x = non_survey_data['Date'].map(datetime.toordinal)
        y = non_survey_data[index]
        spline = interpolate.UnivariateSpline(x, y, s=0.8) # Increase s value for more smoothing
        xs = np.linspace(x.min(), x.max(), 500)
        plt.plot([datetime.fromordinal(int(x)) for x in xs], spline(xs), label='Spline Fit', color='green')
    
    plt.axvspan(survey_start, survey_end, color='grey', alpha=0.3, label='Survey Period')
    
    # Highlight the closest date with >99% clear sky pixels - For downloading full image for SNAP Biophysical Properties analysis
    if closest_date_99:
        plt.scatter(pd.to_datetime(closest_date_99), df[df['Date'] == pd.to_datetime(closest_date_99)][index], color='yellow', label='Closest Date >99% Clear Sky', zorder=5)

    plt.title(f'{index} by Date for Site: {site_name} ({year})')
    plt.xlabel('Date')
    plt.ylabel(index)
    plt.grid(True)
    plt.legend()
    
    months = pd.date_range(start=f'{year}-04-01', end=f'{year}-10-31', freq='MS').strftime('%b')
    plt.xticks(pd.date_range(start=f'{year}-04-01', end=f'{year}-10-31', freq='MS'), months)
    
    plot_path = os.path.join(os.path.dirname(shapefile_path), f'{site_name}_{year}_{index}.png')
    plt.savefig(plot_path)
    plt.close()

# Generate scatterplots for each site and year
for (site_name, year), df in all_data_frames.items():
    survey_start, survey_end = survey_ranges[year]
    closest_date_99 = next((entry['Date'] for entry in closest_dates_all_sites if entry['Site'] == site_name and entry['Year'] == year and entry['Threshold'] == 99), None)
    plot_indices(df, site_name, year, 'NDVI', survey_start, survey_end, closest_date_99)
    plot_indices(df, site_name, year, 'LAI', survey_start, survey_end, closest_date_99)
